<a href="https://colab.research.google.com/github/SangHyeok-Lee1227/petitions/blob/master/p01_%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B3%B4%EA%B8%B0_%EB%B0%8F_%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 출력데이터가 지저분하게 보이지 않도록
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install plotnine
!pip install --upgrade pandas

     |████████████████████████████████| 10.1MB 2.7MB/s 
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype # 그래프의 값을 정렬해서 보기위해
import numpy as np

print(pd.__version__)
print(np.__version__)

1.0.5
1.18.5


In [ ]:
!pip install statsmodels

In [2]:
from plotnine import *

In [5]:
# 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증을 합니다.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
url = 'https://drive.google.com/file/d/16j4sQKllUwAVdQEezK5pCXuatglvtddW/view?usp=sharing'
id = '16j4sQKllUwAVdQEezK5pCXuatglvtddW'
print(id)


16j4sQKllUwAVdQEezK5pCXuatglvtddW


In [6]:
# 크롤링해 온 국민청원 데이터를 판다스를 통해 읽어온다.
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('petition.csv')  

In [7]:
petitions = pd.read_csv('petition.csv', index_col=0, \
                        parse_dates=['start', 'end'])
# 데이터의 크기가 어느정도인지 본다.
petitions.shape

(377756, 7)

In [9]:
petitions.head(10)
petitions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377756 entries, 21 to 492043
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   start     377756 non-null  datetime64[ns]
 1   end       377756 non-null  datetime64[ns]
 2   answered  377756 non-null  int64         
 3   votes     377756 non-null  int64         
 4   category  377756 non-null  object        
 5   title     377756 non-null  object        
 6   content   377755 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 23.1+ MB


In [17]:
petitions.columns
petitions.head(5)
petitions.loc[petitions["votes"]>200000].sort_values(by="votes",ascending=False)

,start,end,answered,votes,category,title,content
article_id,,,,,,,
408609,2018-10-17,2018-11-16,0,1192049,안전/환경,강서구 피시방 살인 사건. 또 심신미약 피의자입니다.,2018년 10월 14일 엊그제 일어난 강서구 피시방 살인사건에 대한 청원입니다.\...
269548,2018-06-13,2018-07-13,0,714875,외교/통일/국방,"제주도 불법 난민 신청 문제에 따른 난민법, 무사증 입국, 난민신청허가 폐지/개헌 ...",2012년 난민법 제정으로 인해 외국인은 한달 무비자로 입국할 수 있으나 난민신청자...
10949,2017-09-06,2017-12-05,1,615354,미래,조두순 출소반대,제발 조두순 재심다시해서 무기징역으로 해야됩니다!!!
142600,2018-02-19,2018-03-21,1,614127,문화/예술/체육/언론,"김보름, 박지우 선수의 자격박탈과 적폐 빙상연맹의 엄중 처벌을 청원합니다","오늘 여자 단체전 팀추월에서 김보름, 박지우 선수는 팀전인데도 불구하고 개인의 영달..."
230552,2018-05-11,2018-06-10,1,419006,인권/성평등,여성도 대한민국 국민입니다. 성별 관계없는 국가의 보호를 요청합니다.,최근 홍대 누드크로키 모델의 불법촬영 사건이 있었습니다.\n사건은 굉장히 빠르게 처...
...,...,...,...,...,...,...,...
134590,2018-02-08,2018-03-10,1,207772,경제민주화,경제민주화,김상조 공정거래위원장이 경제정의 구현을 위한 정책들에 대해서 적극적인 지지를 보냅...
273112,2018-06-17,2018-07-17,0,207275,외교/통일/국방,남편선교사가 안티폴로감옥에 있습니다. (필리핀),저는 기독교대한성결교회 파송 배순영선교사입니다.\n남편(백영모선교사)과 저는 필리핀...
436901,2018-11-09,2018-12-09,0,206447,인권/성평등,부하 여군을 강간한 두 명의 해군 간부를 처벌해주십시오.,2010년 9월경 해군 중위가 직속상관에게 상습적 강간과 강제 추행을 당한 사건이 ...


In [28]:
petitions.describe(include=np.object)
petitions["answer"] = (petitions["votes"]>200000) == 1
petitions[petitions["answer"] == 1]
petitions.shape

(377756, 8)

In [32]:
petitions.columns
petitions["duration"] = petitions["end"] - petitions["start"]
petitions.head(3)
petitions["duration"].value_counts()

30 days    366556
90 days      6754
7 days       2436
15 days      1204
60 days       806
Name: duration, dtype: int64

In [58]:
category = petitions["category"].value_counts().reset_index()
category.columns = ['category', 'counts']
category = category.rename(columns={"category":"cate", "counts":"cnt"})
category.sort_values(by="cnt", ascending=False)

,cate,cnt
0,정치개혁,59020
1,기타,46449
2,인권/성평등,33738
3,안전/환경,29196
4,교통/건축/국토,26910
5,외교/통일/국방,25705
6,육아/교육,24861
7,보건복지,23608
8,일자리,22111
9,행정,19373


In [63]:
petitions_unique=pd.pivot_table(petitions,index="category", aggfunc=np.sum)
petitions_unique = petitions_unique.sort_values(by='votes',ascending=False).reset_index()
petitions_unique

,category,answer,answered,votes
0,인권/성평등,23.0,9,12225998
1,안전/환경,10.0,2,6512799
2,정치개혁,7.0,2,5686172
3,기타,7.0,1,4720310
4,육아/교육,3.0,1,4420589
5,보건복지,3.0,2,4269399
6,문화/예술/체육/언론,7.0,4,4086382
7,외교/통일/국방,4.0,1,3637926
8,교통/건축/국토,3.0,2,2834727
9,반려동물,4.0,0,2023905


In [68]:
petitions_unique2 = petitions.groupby(by="category")['answer','answered','votes'].sum().reset_index().sort_values(by='votes',ascending=False)
petitions_unique2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,category,answer,answered,votes
12,인권/성평등,23.0,9,12225998
9,안전/환경,10.0,2,6512799
15,정치개혁,7.0,2,5686172
2,기타,7.0,1,4720310
11,육아/교육,3.0,1,4420589
7,보건복지,3.0,2,4269399
4,문화/예술/체육/언론,7.0,4,4086382
10,외교/통일/국방,4.0,1,3637926
1,교통/건축/국토,3.0,2,2834727
6,반려동물,4.0,0,2023905


In [71]:
petitions.pivot_table(index='duration', columns='category', aggfunc='sum').astype(int)

answer                   ...    votes                            
category  경제민주화 교통/건축/국토 기타 농산어촌  ...      일자리 저출산/고령화대책     정치개혁       행정
duration                          ...                                     
7 days        0        0  0    0  ...     4491         5    27809    12032
15 days       0        0  0    0  ...     6853        19      419     1232
30 days       2        3  7    0  ...  1076584    174793  5616274  1646231
60 days       0        0  0    0  ...     1938         4      672      329
90 days       0        0  0    0  ...    24107        20    40998     6031

[5 rows x 51 columns]

In [73]:
import seaborn as sns

In [75]:
sns.barplot(petitions)

TypeError: ignored